In [ ]:
---------------------------------------------------------------------------------

In [2]:
#final ver牛逼
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import os
import re
import requests
import time

# 設定帳號與密碼
username_str = 'XXXXXXXXXXXXXXX'
password_str = 'XXXXXXXXXXXXXXX'
download_dir = r"C:\Users\N000189549\Desktop\Python\tokenizer"

# 配置 ChromeDriver
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "plugins.always_open_pdf_externally": False,
    "download.prompt_for_download": True,
    "directory_upgrade": True,
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(service=service, options=options)

try:
    driver.get('https://www.naturalgasintel.com/account/login/')
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, 'id_username')))
    
    # 登入
    driver.find_element(By.ID, 'id_username').send_keys(username_str)
    driver.find_element(By.ID, 'field-password').send_keys(password_str)
    driver.find_element(By.XPATH, "//button[text()='Sign In']").click()
    
    # 進入 Daily Gas Price Index 頁面
    driver.get('https://www.naturalgasintel.com/news/daily-gas-price-index/')
    
    # 接受 cookie
    cookie_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))
    )
    cookie_button.click()
    print("已接受 cookie")
    
    # 點擊 "View Issue" 按鈕
    view_issue_button = WebDriverWait(driver, 30).until(
        EC.element_to_be_clickable((By.LINK_TEXT, "View Issue"))
    )
    view_issue_button.click()
    print("已點擊 'View Issue' 按鈕")
    
    # 等待頁面跳轉
    time.sleep(5)
    current_url = driver.current_url
    print(f"跳轉後的 URL: {current_url}")
    
    # 提取日期
    pattern = r'dg(\d{8})'
    match = re.search(pattern, current_url)
    
    if match:
        date_str = match.group(1)
        file_name = f"NGI daily index_{date_str}.pdf"
        print(f"將下載的文件命名為: {file_name}")
        
        # 獲取 cookies
        cookies = driver.get_cookies()
        
        # 使用 requests 獲取 session
        session = requests.Session()
        for cookie in cookies:
            session.cookies.set(cookie['name'], cookie['value'])

        # 使用 session 下載文件的header 避免403
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Referer': 'https://www.naturalgasintel.com/news/daily-gas-price-index/',
        }

        # 使用 session 下載文件
        response = session.get(current_url,headers=headers)
        
        if response.status_code == 200:
            with open(os.path.join(download_dir, file_name), 'wb') as f:
                f.write(response.content)
            print(f"文件已下載並命名為: {file_name}")
        else:
            print(f"無法下載文件，HTTP 狀態碼: {response.status_code}")
    else:
        print("未能提取 URL 中的日期")

except Exception as e:
    print(f"發生錯誤: {e}")

finally:
    driver.quit()


已接受 cookie
已點擊 'View Issue' 按鈕
跳轉後的 URL: https://www.naturalgasintel.com/protected_documents/dg20250207/
將下載的文件命名為: NGI daily index_20250207.pdf
文件已下載並命名為: NGI daily index_20250207.pdf
